# Workbook for Solving a TSP

In [1]:
import veroviz as vrv

In [2]:
vrv.checkVersion()

'Your current installed version of veroviz is 0.4.0, the latest version available is 0.4.3. To update to the latest version, type `pip install --upgrade veroviz` at a command-line prompt.'

In [3]:
ORS_API_KEY='5b3ce3597851110001cf62480fb8f5604a0b485b911812eedd4d31cf'

# 1 -- Create some "nodes"

In [4]:
myBoundingRegion = [[43.01770145196991, -78.87840270996095], [42.878480017739044, -78.8756561279297], [42.83569550641454, -78.68270874023439], [42.96596996868038, -78.60717773437501], [43.04430359661548, -78.72528076171876]]
myBoundingRegion

[[43.01770145196991, -78.87840270996095],
 [42.878480017739044, -78.8756561279297],
 [42.83569550641454, -78.68270874023439],
 [42.96596996868038, -78.60717773437501],
 [43.04430359661548, -78.72528076171876]]

In [5]:
nodesDF = vrv.generateNodes(nodeType = 'customer',
                            nodeName = 'cust',
                            numNodes = 10,
                            incrementName = True,
                            nodeDistrib = 'uniformBB',
                            nodeDistribArgs = {'boundingRegion': myBoundingRegion},
                            snapToRoad = True,
                            dataProvider     = 'ORS-online',
                            dataProviderArgs = {'APIkey' : ORS_API_KEY})
nodesDF

,id,lat,lon,altMeters,nodeName,nodeType,popupText,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText,elevMeters
0,1,42.970124,-78.639963,0,cust1,customer,1,glyphicon,info-sign,blue,1,pin,blue,1,None
1,2,42.903435,-78.648069,0,cust2,customer,2,glyphicon,info-sign,blue,2,pin,blue,2,None
2,3,42.932958,-78.783677,0,cust3,customer,3,glyphicon,info-sign,blue,3,pin,blue,3,None
3,4,42.908109,-78.816682,0,cust4,customer,4,glyphicon,info-sign,blue,4,pin,blue,4,None
4,5,42.960335,-78.620077,0,cust5,customer,5,glyphicon,info-sign,blue,5,pin,blue,5,None
5,6,42.887163,-78.759636,0,cust6,customer,6,glyphicon,info-sign,blue,6,pin,blue,6,None
6,7,42.897936,-78.664912,0,cust7,customer,7,glyphicon,info-sign,blue,7,pin,blue,7,None
7,8,42.864326,-78.691514,0,cust8,customer,8,glyphicon,info-sign,blue,8,pin,blue,8,None
8,9,42.930827,-78.716313,0,cust9,customer,9,glyphicon,info-sign,blue,9,pin,blue,9,None
9,10,42.904578,-78.814901,0,cust10,customer,10,glyphicon,info-sign,blue,10,pin,blue,10,None


In [6]:
len(nodesDF)

10

In [7]:
# View our nodes and bounding region:
vrv.createLeaflet(nodes = nodesDF, boundingRegion=myBoundingRegion)

In [8]:
# 2. Get time and distance matrices
[timeSec, distMeters] = vrv.getTimeDist2D(nodes = nodesDF,
                                          routeType = 'euclidean2D',
                                          speedMPS = vrv.convertSpeed(25, 'miles', 'hr', 'meters', 'second'))

In [9]:
#timeSec
timeSec[1,2]

665.5410265267658

In [10]:
#distance
distMeters[1,2]

7438.068025212398

In [11]:
# Optional, just to view the data in a table:
vrv.convertMatricesDictionaryToDataframe(timeSec)

,1,2,3,4,5,6,7,8,9,10
1,0.000000,665.541027,1112.507110,1430.289714,174.766120,1201.745527,740.339971,1117.056754,680.731828,1434.196895
2,665.541027,0.000000,1033.270508,1232.970325,601.417845,831.272043,134.677187,501.975100,568.086677,1219.168216
3,1112.507110,1033.270508,0.000000,345.178975,1225.275935,487.926971,934.937386,958.676864,492.492465,362.790988
4,1430.289714,1232.970325,345.178975,0.000000,1526.936567,466.002382,1113.668641,1013.156724,767.242873,37.433730
5,174.766120,601.417845,1225.275935,1526.936567,0.000000,1252.356093,701.408613,1087.753901,761.521468,1527.132983
6,1201.745527,831.272043,487.926971,466.002382,1252.356093,0.000000,700.551226,547.319704,537.202536,439.432121
7,740.339971,134.677187,934.937386,1113.668641,701.408613,700.551226,0.000000,386.562637,497.922329,1098.071380
8,1117.056754,501.975100,958.676864,1013.156724,1087.753901,547.319704,386.562637,0.000000,685.424167,986.692683
9,680.731828,568.086677,492.492465,767.242873,761.521468,537.202536,497.922329,685.424167,0.000000,766.069987
10,1434.196895,1219.168216,362.790988,37.433730,1527.132983,439.432121,1098.071380,986.692683,766.069987,0.000000


In [12]:
def tsp_cost(route, costDict):
    cost = 0
    
    i = route[0]
    for j in route[1:]:
        cost += costDict[i,j]
        i = j
        
    cost += costDict[i, route[0]]
    
    return cost

In [13]:
def tsp_neighbor(route):
    """
        This function computes a "subtour" solution to a TSP.
        
        Inputs
        ------
        route: list of Integer, indicating the node where the salesperson begins (and ends) the route
        
        Returns
        -------
        A random list of nodeIDs specifying another TSP route.
        """
    #import random
    from random import randint 
    
    #a = random.randint(0,len(route)-3)
    a = randint(0,len(route)-3)
    #b = random.randint(a+1, len(route)-2)
    b = randint(a+1, len(route)-2)
    
    newRoute = []
    newRoute.extend(route[0:a])
    
    subtour = route[a:b+1]
    subtour.reverse()
    newRoute.extend(subtour)
    
    newRoute.extend(route[b+1:len(route)-1])
    
    newRoute.append(newRoute[0])
    
    return newRoute

In [14]:
#Example of subtour
a=[1,5,8,9,2,4,3,7,6]
tsp_neighbor(a)

[1, 9, 8, 5, 2, 4, 3, 7, 1]

In [15]:
def solve_tsp_nn(startNode, costDict, nodesDF): 
    """
    This function computes a "nearest neighbor" solution to a TSP.
    
    Inputs
    ------
    startNode: Integer, indicating the node where the salesperson begins (and ends) the route
    
    costDict: VeRoViz time or distance dictionary.
    
    nodesDF: VeRoViz nodes dataframe
    
    Returns
    -------
    An ordered list of nodeIDs specifying a TSP route.
    """
    
    # Solve the TSP with a "nearest neighbor" heuristic
    nn_route = []

    # Start our route by visiting the startNode
    nn_route.append(startNode)

    # Initialize a list of unvisited nodes
    unvisitedNodes = list(nodesDF[nodesDF['id'] != startNode]['id'])

    # Let i represent our "current" location:
    i = startNode

    while len(unvisitedNodes) > 0:
        # Initialize minTime to a huge value
        minTime = float('inf')

        # Find the nearest unvisited node to our current node:
        for j in unvisitedNodes:
            if (costDict[i,j] < minTime):
                nextNode = j
                minTime = costDict[i,j]

        # Update our salesperson's location
        i = nextNode

        # Append nextNode to our route:
        nn_route.append(nextNode)

        # Remove nextNode from our list of unvisitedNodes:
        unvisitedNodes.remove(nextNode)

    nn_route.append(startNode)

    return nn_route    

In [16]:
def solveTSP_SA(nodesDF, costDict, timeLimit):
    import time
    from random import random
    import math as m
    
    runTime=0
    start_time = time.time()
    startnode=1
    T0=4000
    I=3
    d=5
    Tfinal=2020
    cutoffTime=timeLimit
    
    # Initialize by calling your nearest neighbor function
    X0 = solve_tsp_nn(startnode,costDict,nodesDF)
    Z=tsp_cost(X0,costDict)
    print('Intial Solution : X0 =',X0,'Z0 =',Z)
    
    # Simulated annealing (SA) procedure
    Xcur = tsp_neighbor(X0)
    Zcur = tsp_cost(Xcur,costDict)    
    Tcur = T0
    print('Current Solution : Xcur =',Xcur,'Zcur =',Zcur)
    Xbest = Xcur
    Zbest = Zcur
    print('Best known Solution : Xbest =',Xbest,'Zbest =',Zbest)
    
    def solveTSP_SA2(I,X0,Z,Xcur,Zcur,Xbest,Zbest,Tcur,d,Tfinal,runTime):
        print('Tcur=',Tcur)
        for i in range(1,I+1):
            #Generate a neighbor solution, Xcount
            # call tsp_neighbor(route)
            # ...
            # ...
            Xcount=tsp_neighbor(X0)
            Z = tsp_cost(Xcount,costDict)
            print('i=',i)
            print('Xcount=',Xcount,'Zcount=',Z)

            if (Z<Zcur):
                print('Z-Zcur=',Z-Zcur)
                Xcur = Xcount
                Zcur = Z
                print('Xcur=',Xcur,'Zcur=',Z)


            else:
                C=Z-Zcur
                print('C=',C)
                if(random()<=m.exp(-C/Tcur)):
                    Xcur = Xcount
                    Zcur = Z
                    print('Xcur=',Xcur,'Zcur=',Zcur)


            if(Z < Zbest):
                print('Z-Zbest=',Z-Zbest)
                Zbest = Z
                Xbest = Xcur
                print('Xbest=',Xbest,'Zbest=',Zbest)

        Tcur=Tcur-d
        #print('Tcur=',Tcur)
        print('Xbest=',Xbest,'Zbest=',Zbest)
            
        runTime=time.time()-start_time
        print(runTime)
        if(Tcur<Tfinal or runTime>=timeLimit):
            if(Tcur<Tfinal):
                print('Tcur=',Tcur,'< Tfinal=',Tfinal)
                print('runTime =',runTime,',cutoffTime =',cutoffTime)
            elif(runTime>=cutoffTime):
                print('runTime =',runTime,'> cutoffTime =',cutoffTime)
                print('Tcur=',Tcur,',Tfinal=',Tfinal)
            return Xbest

        return solveTSP_SA2(I,X0,Z,Xcur,Zcur,Xbest,Zbest,Tcur,d,Tfinal,runTime)
    
    
    
    XBEST=solveTSP_SA2(I,X0,Z,Xcur,Zcur,Xbest,Zbest,Tcur,d,Tfinal,runTime)
    
    totalruntime=time.time()-start_time
    print('totaltime=',totalruntime)
    
    # SA produces a "best" solution, X_best.
    # X_best is just a sequence of node numbers,
    # starting from and returning to a "home" location.
    # For example, Xbest = [1, 5, 3, 4, 2, 1].
    
    # Your function should return a VeRoViz "assignments" dataframe.
    # Fortunately, VeRoViz provides a function to make this very easy:
    assignmentsDF = vrv.createAssignmentsFromNodeSeq2D(
        nodeSeq          = XBEST,
        nodes            = nodesDF,
        objectID         = 'Blue Car',
        modelFile        = 'veroviz/models/car_blue.gltf',
        modelScale       = 100,
        modelMinPxSize   = 75,
        routeType        = 'fastest',
        speedMPS         = None,
        leafletColor     = 'blue',
        leafletWeight    = 3,
        leafletStyle     = 'dashed',
        leafletOpacity   = 0.8,
        useArrows        = True,
        cesiumColor      = 'blue',
        cesiumWeight     = 3,
        cesiumStyle      = 'solid',
        cesiumOpacity    = 0.8,
        dataProvider     = 'ORS-online',
        dataProviderArgs = {'APIkey' : ORS_API_KEY})

    # See https://veroviz.org/docs/veroviz.createAssignments.html#veroviz.createAssignments.createAssignmentsFromNodeSeq2D
    # for more info on this function.
    
    return assignmentsDF
    #return Xbest

In [17]:
#%time
X_best=solveTSP_SA(nodesDF,timeSec,20)
X_best

Intial Solution : X0 = [1, 5, 2, 7, 8, 6, 10, 4, 3, 9, 1] Z0 = 3840.0126121154135
Current Solution : Xcur = [1, 5, 2, 7, 8, 6, 10, 4, 9, 3, 1] Zcur = 4693.851791738705
Best known Solution : Xbest = [1, 5, 2, 7, 8, 6, 10, 4, 9, 3, 1] Zbest = 4693.851791738705
Tcur= 4000
i= 1
Xcount= [1, 5, 2, 7, 8, 3, 4, 10, 6, 9, 1] Zcount= 4296.079843175209
Z-Zcur= -397.77194856349615
Xcur= [1, 5, 2, 7, 8, 3, 4, 10, 6, 9, 1] Zcur= 4296.079843175209
Z-Zbest= -397.77194856349615
Xbest= [1, 5, 2, 7, 8, 3, 4, 10, 6, 9, 1] Zbest= 4296.079843175209
i= 2
Xcount= [6, 8, 7, 2, 5, 1, 10, 4, 3, 9, 6] Zcount= 4691.248094091451
C= 395.16825091624196
Xcur= [6, 8, 7, 2, 5, 1, 10, 4, 3, 9, 6] Zcur= 4691.248094091451
i= 3
Xcount= [1, 2, 5, 7, 8, 6, 10, 4, 3, 9, 1] Zcount= 4897.518943782848
C= 206.2708496913965
Xcur= [1, 2, 5, 7, 8, 6, 10, 4, 3, 9, 1] Zcur= 4897.518943782848
Xbest= [1, 5, 2, 7, 8, 3, 4, 10, 6, 9, 1] Zbest= 4296.079843175209
0.014290094375610352
Tcur= 3995
i= 1
Xcount= [1, 5, 2, 7, 6, 8, 10, 4, 3, 9, 1]

Xcur= [1, 5, 2, 7, 8, 6, 10, 4, 9, 3, 1] Zcur= 4693.851791738705
i= 2
Xcount= [1, 5, 2, 7, 8, 6, 10, 4, 9, 3, 1] Zcount= 4693.851791738705
C= 0.0
Xcur= [1, 5, 2, 7, 8, 6, 10, 4, 9, 3, 1] Zcur= 4693.851791738705
i= 3
Xcount= [1, 5, 2, 7, 8, 6, 10, 4, 9, 3, 1] Zcount= 4693.851791738705
C= 0.0
Xcur= [1, 5, 2, 7, 8, 6, 10, 4, 9, 3, 1] Zcur= 4693.851791738705
Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 3840.0126121154135
0.2455151081085205
Tcur= 3895
i= 1
Xcount= [8, 7, 2, 5, 1, 6, 10, 4, 3, 9, 8] Zcount= 4499.130773952685
Z-Zcur= -194.72101778602064
Xcur= [8, 7, 2, 5, 1, 6, 10, 4, 3, 9, 8] Zcur= 4499.130773952685
i= 2
Xcount= [1, 5, 2, 7, 8, 6, 3, 4, 10, 9, 1] Zcount= 4162.084984521761
Z-Zcur= -337.04578943092383
Xcur= [1, 5, 2, 7, 8, 6, 3, 4, 10, 9, 1] Zcur= 4162.084984521761
i= 3
Xcount= [1, 5, 2, 7, 8, 6, 9, 3, 4, 10, 1] Zcount= 4691.248094091451
C= 529.1631095696903
Xcur= [1, 5, 2, 7, 8, 6, 9, 3, 4, 10, 1] Zcur= 4691.248094091451
Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 38

Tcur= 3780
i= 1
Xcount= [1, 5, 2, 7, 8, 6, 10, 4, 9, 3, 1] Zcount= 4693.851791738705
C= 708.9263586634124
Xcur= [1, 5, 2, 7, 8, 6, 10, 4, 9, 3, 1] Zcur= 4693.851791738705
i= 2
Xcount= [1, 5, 7, 2, 8, 6, 10, 4, 3, 9, 1] Zcount= 4055.4158424620096
Z-Zcur= -638.4359492766957
Xcur= [1, 5, 7, 2, 8, 6, 10, 4, 3, 9, 1] Zcur= 4055.4158424620096
i= 3
Xcount= [1, 5, 2, 7, 9, 3, 4, 10, 6, 8, 1] Zcount= 4387.697229289603
C= 332.281386827593
Xcur= [1, 5, 2, 7, 9, 3, 4, 10, 6, 8, 1] Zcur= 4387.697229289603
Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 3840.0126121154135
0.45178699493408203
Tcur= 3775
i= 1
Xcount= [1, 5, 2, 7, 9, 3, 4, 10, 6, 8, 1] Zcount= 4387.697229289603
C= 0.0
Xcur= [1, 5, 2, 7, 9, 3, 4, 10, 6, 8, 1] Zcur= 4387.697229289603
i= 2
Xcount= [1, 5, 2, 7, 8, 4, 10, 6, 3, 9, 1] Zcount= 4448.597627137577
C= 60.90039784797409
Xcur= [1, 5, 2, 7, 8, 4, 10, 6, 3, 9, 1] Zcur= 4448.597627137577
i= 3
Xcount= [1, 5, 2, 7, 8, 6, 3, 4, 10, 9, 1] Zcount= 4162.084984521761
Z-Zcur= -286.5126426158

Xcount= [1, 8, 7, 2, 5, 6, 10, 4, 3, 9, 1] Zcount= 5487.339633444994
C= 796.0915393535433
Xcur= [1, 8, 7, 2, 5, 6, 10, 4, 3, 9, 1] Zcur= 5487.339633444994
Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 3840.0126121154135
1.1391429901123047
Tcur= 3660
i= 1
Xcount= [1, 5, 2, 7, 8, 6, 10, 4, 9, 3, 1] Zcount= 4693.851791738705
Z-Zcur= -793.4878417062891
Xcur= [1, 5, 2, 7, 8, 6, 10, 4, 9, 3, 1] Zcur= 4693.851791738705
i= 2
Xcount= [1, 8, 7, 2, 5, 6, 10, 4, 3, 9, 1] Zcount= 5487.339633444994
C= 793.4878417062891
Xcur= [1, 8, 7, 2, 5, 6, 10, 4, 3, 9, 1] Zcur= 5487.339633444994
i= 3
Xcount= [7, 2, 5, 1, 8, 6, 10, 4, 3, 9, 7] Zcount= 4387.697229289603
Z-Zcur= -1099.642404155392
Xcur= [7, 2, 5, 1, 8, 6, 10, 4, 3, 9, 7] Zcur= 4387.697229289603
Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 3840.0126121154135
1.1396269798278809
Tcur= 3655
i= 1
Xcount= [1, 5, 2, 7, 8, 6, 10, 3, 4, 9, 1] Zcount= 4440.120278470147
C= 52.423049180544695
Xcur= [1, 5, 2, 7, 8, 6, 10, 3, 4, 9, 1] Zcur= 4440.120278470

Xcount= [5, 1, 2, 7, 8, 6, 10, 4, 3, 9, 5] Zcount= 3984.925433075293
Z-Zcur= -455.1948453948544
Xcur= [5, 1, 2, 7, 8, 6, 10, 4, 3, 9, 5] Zcur= 3984.925433075293
Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 3840.0126121154135
1.5980820655822754
Tcur= 3480
i= 1
Xcount= [1, 5, 2, 7, 8, 6, 10, 4, 9, 3, 1] Zcount= 4693.851791738705
C= 708.9263586634124
Xcur= [1, 5, 2, 7, 8, 6, 10, 4, 9, 3, 1] Zcur= 4693.851791738705
i= 2
Xcount= [1, 5, 2, 7, 8, 6, 10, 3, 4, 9, 1] Zcount= 4440.120278470147
Z-Zcur= -253.73151326855805
Xcur= [1, 5, 2, 7, 8, 6, 10, 3, 4, 9, 1] Zcur= 4440.120278470147
i= 3
Xcount= [1, 6, 8, 7, 2, 5, 10, 4, 3, 9, 1] Zcount= 5954.692881203504
C= 1514.5726027333567
Xcur= [1, 6, 8, 7, 2, 5, 10, 4, 3, 9, 1] Zcur= 5954.692881203504
Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 3840.0126121154135
1.6000010967254639
Tcur= 3475
i= 1
Xcount= [1, 5, 3, 4, 10, 6, 8, 7, 2, 9, 1] Zcount= 4539.464914342187
Z-Zcur= -1415.2279668613173
Xcur= [1, 5, 3, 4, 10, 6, 8, 7, 2, 9, 1] Zcur= 4539.4

...

Xcur= [3, 4, 10, 6, 8, 7, 2, 5, 1, 9, 3] Zcur= 3840.0126121154135
i= 3
Xcount= [1, 5, 2, 6, 8, 7, 10, 4, 3, 9, 1] Zcount= 5195.246726891654
C= 1355.2341147762409
Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 3840.0126121154135
3.370744228363037
Tcur= 2415
i= 1
Xcount= [6, 8, 7, 2, 5, 1, 10, 4, 3, 9, 6] Zcount= 4691.248094091451
C= 851.2354819760376
Xcur= [6, 8, 7, 2, 5, 1, 10, 4, 3, 9, 6] Zcur= 4691.248094091451
i= 2
Xcount= [1, 5, 2, 7, 4, 10, 6, 8, 3, 9, 1] Zcount= 5180.6165054757585
C= 489.36841138430736
Xcur= [1, 5, 2, 7, 4, 10, 6, 8, 3, 9, 1] Zcur= 5180.6165054757585
i= 3
Xcount= [1, 5, 2, 7, 8, 6, 10, 9, 3, 4, 1] Zcount= 5318.206755257309
C= 137.59024978155048
Xcur= [1, 5, 2, 7, 8, 6, 10, 9, 3, 4, 1] Zcur= 5318.206755257309
Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 3840.0126121154135
3.3727450370788574
Tcur= 2410
i= 1
Xcount= [1, 5, 2, 7, 8, 6, 10, 9, 3, 4, 1] Zcount= 5318.206755257309
C= 0.0
Xcur= [1, 5, 2, 7, 8, 6, 10, 9, 3, 4, 1] Zcur= 5318.206755257309
i= 2
Xcount= 

Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 3840.0126121154135
3.5740721225738525
Tcur= 2285
i= 1
Xcount= [9, 3, 4, 10, 6, 8, 7, 2, 5, 1, 9] Zcount= 3840.012612115414
Z-Zcur= -144.9128209598789
Xcur= [9, 3, 4, 10, 6, 8, 7, 2, 5, 1, 9] Zcur= 3840.012612115414
i= 2
Xcount= [3, 4, 10, 6, 8, 7, 2, 5, 1, 9, 3] Zcount= 3840.0126121154135
Z-Zcur= -4.547473508864641e-13
Xcur= [3, 4, 10, 6, 8, 7, 2, 5, 1, 9, 3] Zcur= 3840.0126121154135
i= 3
Xcount= [1, 5, 2, 7, 8, 10, 6, 4, 3, 9, 1] Zcount= 4707.954242583282
C= 867.9416304678689
Xcur= [1, 5, 2, 7, 8, 10, 6, 4, 3, 9, 1] Zcur= 4707.954242583282
Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 3840.0126121154135
3.658701181411743
Tcur= 2280
i= 1
Xcount= [10, 6, 8, 7, 2, 5, 1, 4, 3, 9, 10] Zcount= 5318.206755257309
C= 610.2525126740265
Xcur= [10, 6, 8, 7, 2, 5, 1, 4, 3, 9, 10] Zcur= 5318.206755257309
i= 2
Xcount= [1, 5, 9, 3, 4, 10, 6, 8, 7, 2, 1] Zcount= 3984.925433075293
Z-Zcur= -1333.281322182016
Xcur= [1, 5, 9, 3, 4, 10, 6, 8, 7, 2, 1] Zcu

Xcount= [1, 5, 2, 7, 8, 6, 10, 3, 4, 9, 1] Zcount= 4440.120278470147
Z-Zcur= -1047.2193549748472
Xcur= [1, 5, 2, 7, 8, 6, 10, 3, 4, 9, 1] Zcur= 4440.120278470147
i= 2
Xcount= [2, 5, 1, 7, 8, 6, 10, 4, 3, 9, 2] Zcount= 4333.030245339506
Z-Zcur= -107.09003313064113
Xcur= [2, 5, 1, 7, 8, 6, 10, 4, 3, 9, 2] Zcur= 4333.030245339506
i= 3
Xcount= [1, 5, 2, 7, 8, 6, 10, 4, 9, 3, 1] Zcount= 4693.851791738705
C= 360.8215463991992
Xcur= [1, 5, 2, 7, 8, 6, 10, 4, 9, 3, 1] Zcur= 4693.851791738705
Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 3840.0126121154135
3.7781479358673096
Tcur= 2200
i= 1
Xcount= [10, 6, 8, 7, 2, 5, 1, 4, 3, 9, 10] Zcount= 5318.206755257309
C= 624.3549635186037
Xcur= [10, 6, 8, 7, 2, 5, 1, 4, 3, 9, 10] Zcur= 5318.206755257309
i= 2
Xcount= [1, 5, 2, 7, 8, 6, 10, 9, 3, 4, 1] Zcount= 5318.206755257309
C= 0.0
Xcur= [1, 5, 2, 7, 8, 6, 10, 9, 3, 4, 1] Zcur= 5318.206755257309
i= 3
Xcount= [1, 5, 2, 7, 8, 6, 10, 4, 9, 3, 1] Zcount= 4693.851791738705
Z-Zcur= -624.3549635186037
Xcur

C= 80.94708818281924
Xcur= [1, 5, 2, 7, 8, 6, 9, 3, 4, 10, 1] Zcur= 4691.248094091451
Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 3840.0126121154135
3.9838879108428955
Tcur= 2085
i= 1
Xcount= [1, 5, 2, 7, 8, 6, 9, 3, 4, 10, 1] Zcount= 4691.248094091451
C= 0.0
Xcur= [1, 5, 2, 7, 8, 6, 9, 3, 4, 10, 1] Zcur= 4691.248094091451
i= 2
Xcount= [4, 10, 6, 8, 7, 2, 5, 1, 3, 9, 4] Zcount= 4693.851791738705
C= 2.6036976472541937
Xcur= [4, 10, 6, 8, 7, 2, 5, 1, 3, 9, 4] Zcur= 4693.851791738705
i= 3
Xcount= [1, 5, 2, 7, 4, 10, 6, 8, 3, 9, 1] Zcount= 5180.6165054757585
C= 486.76471373705317
Xcur= [1, 5, 2, 7, 4, 10, 6, 8, 3, 9, 1] Zcur= 5180.6165054757585
Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 3840.0126121154135
3.9846510887145996
Tcur= 2080
i= 1
Xcount= [10, 6, 8, 7, 2, 5, 1, 4, 3, 9, 10] Zcount= 5318.206755257309
C= 137.59024978155048
Xcur= [10, 6, 8, 7, 2, 5, 1, 4, 3, 9, 10] Zcur= 5318.206755257309
i= 2
Xcount= [3, 4, 10, 6, 8, 7, 2, 5, 1, 9, 3] Zcount= 3840.0126121154135
Z-Zcur= -1

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,ganttColor,popupText,startElevMeters,endElevMeters,wayname,waycategory,surface,waytype,steepness,tollway
0,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,0.000000,42.970124,-78.639963,0,5.895919,...,darkgray,None,225.8,225.7,-,No category,Asphalt,Street,0,False
1,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,5.895919,42.970408,-78.639858,0,24.800000,...,darkgray,None,225.7,225.4,-,No category,Asphalt,Street,0,False
2,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,24.800000,42.971338,-78.640075,0,26.658000,...,darkgray,None,225.4,225.4,"Main Street, NY 5",No category,Asphalt,State Road,0,False
3,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,26.658000,42.971260,-78.640441,0,30.446964,...,darkgray,None,225.4,225.2,"Main Street, NY 5",No category,Asphalt,State Road,0,False
4,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,30.446964,42.971100,-78.641187,0,30.818380,...,darkgray,None,225.2,225.1,"Main Street, NY 5",No category,Asphalt,State Road,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,5208.467360,42.971555,-78.639066,0,5209.897826,...,darkgray,None,225.6,225.7,"Main Street, NY 5",No category,Asphalt,State Road,0,False
1241,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,5209.897826,42.971494,-78.639347,0,5212.658533,...,darkgray,None,225.7,225.8,"Main Street, NY 5",No category,Asphalt,State Road,0,False
1242,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,5212.658533,42.971378,-78.639890,0,5213.600000,...,darkgray,None,225.8,225.8,"Main Street, NY 5",No category,Asphalt,State Road,0,False
1243,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,5213.600000,42.971338,-78.640075,0,5232.504081,...,darkgray,None,225.8,225.5,-,No category,Asphalt,Street,0,False


In [18]:
vrv.createLeaflet(arcs=X_best, nodes=nodesDF)

In [19]:
#%time
X_best=solveTSP_SA(nodesDF,distMeters,2)
X_best

Intial Solution : X0 = [1, 5, 2, 7, 8, 6, 10, 4, 3, 9, 1] Z0 = 42915.87428598486
Current Solution : Xcur = [1, 5, 2, 7, 8, 6, 10, 9, 3, 4, 1] Zcur = 59436.13096879027
Best known Solution : Xbest = [1, 5, 2, 7, 8, 6, 10, 9, 3, 4, 1] Zbest = 59436.13096879027
Tcur= 4000
i= 1
Xcount= [1, 5, 2, 7, 8, 6, 3, 4, 10, 9, 1] Zcount= 46515.3461735434
Z-Zcur= -12920.784795246866
Xcur= [1, 5, 2, 7, 8, 6, 3, 4, 10, 9, 1] Zcur= 46515.3461735434
Z-Zbest= -12920.784795246866
Xbest= [1, 5, 2, 7, 8, 6, 3, 4, 10, 9, 1] Zbest= 46515.3461735434
i= 2
Xcount= [1, 7, 2, 5, 8, 6, 10, 4, 3, 9, 1] Zcount= 57073.206019413454
C= 10557.85984587005
i= 3
Xcount= [1, 5, 2, 7, 8, 6, 10, 4, 9, 3, 1] Zcount= 52458.35723969979
C= 5943.011066156389
Xbest= [1, 5, 2, 7, 8, 6, 3, 4, 10, 9, 1] Zbest= 46515.3461735434
0.0022177696228027344
Tcur= 3995
i= 1
Xcount= [1, 5, 2, 7, 4, 10, 6, 8, 3, 9, 1] Zcount= 57898.42615918303
C= 11383.079985639626
i= 2
Xcount= [1, 5, 2, 10, 6, 8, 7, 4, 3, 9, 1] Zcount= 67064.08736792342
C= 20548.74

Xcount= [1, 5, 3, 4, 10, 6, 8, 7, 2, 9, 1] Zcount= 50732.93378644066
C= 450.7732326002879
i= 3
Xcount= [8, 7, 2, 5, 1, 6, 10, 4, 3, 9, 8] Zcount= 50282.160553840375
C= 0.0
Xcur= [8, 7, 2, 5, 1, 6, 10, 4, 3, 9, 8] Zcur= 50282.160553840375
Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 42915.87428598486
0.3395199775695801
Tcur= 3895
i= 1
Xcount= [1, 5, 2, 7, 8, 6, 9, 3, 4, 10, 1] Zcount= 52429.25838711901
C= 2147.0978332786326
i= 2
Xcount= [1, 5, 2, 7, 8, 6, 9, 3, 4, 10, 1] Zcount= 52429.25838711901
C= 2147.0978332786326
i= 3
Xcount= [1, 5, 2, 7, 8, 6, 10, 4, 9, 3, 1] Zcount= 52458.35723969979
C= 2176.196685859417
Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 42915.87428598486
0.3417367935180664
Tcur= 3890
i= 1
Xcount= [1, 5, 2, 7, 8, 9, 3, 4, 10, 6, 1] Zcount= 50282.160553840375
C= 0.0
Xcur= [1, 5, 2, 7, 8, 9, 3, 4, 10, 6, 1] Zcur= 50282.160553840375
i= 2
Xcount= [1, 5, 2, 7, 8, 6, 10, 9, 3, 4, 1] Zcount= 59436.13096879027
C= 9153.970414949894
Xcur= [1, 5, 2, 7, 8, 6, 10, 9, 3, 4, 

Xcur= [1, 5, 2, 7, 8, 4, 10, 6, 3, 9, 1] Zcur= 49717.40350873325
i= 2
Xcount= [1, 5, 2, 7, 8, 6, 10, 4, 9, 3, 1] Zcount= 52458.35723969979
C= 2740.9537309665393
i= 3
Xcount= [1, 5, 2, 3, 4, 10, 6, 8, 7, 9, 1] Zcount= 53019.21228349585
C= 3301.8087747625977
Xcur= [1, 5, 2, 3, 4, 10, 6, 8, 7, 9, 1] Zcur= 53019.21228349585
Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 42915.87428598486
1.0602688789367676
Tcur= 3740
i= 1
Xcount= [1, 5, 10, 6, 8, 7, 2, 4, 3, 9, 1] Zcount= 66622.92473381666
C= 13603.71245032081
i= 2
Xcount= [1, 5, 2, 9, 3, 4, 10, 6, 8, 7, 1] Zcount= 48425.825659963055
Z-Zcur= -4593.386623532795
Xcur= [1, 5, 2, 9, 3, 4, 10, 6, 8, 7, 1] Zcur= 48425.825659963055
i= 3
Xcount= [1, 5, 2, 7, 8, 6, 10, 9, 3, 4, 1] Zcount= 59436.13096879027
C= 11010.305308827214
Xcur= [1, 5, 2, 7, 8, 6, 10, 9, 3, 4, 1] Zcur= 59436.13096879027
Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 42915.87428598486
1.068741798400879
Tcur= 3735
i= 1
Xcount= [1, 5, 2, 7, 10, 6, 8, 4, 3, 9, 1] Zcount= 61772

Tcur= 3615
i= 1
Xcount= [1, 5, 2, 7, 8, 6, 9, 3, 4, 10, 1] Zcount= 52429.25838711901
C= 9513.38410113415
i= 2
Xcount= [1, 5, 2, 9, 3, 4, 10, 6, 8, 7, 1] Zcount= 48425.825659963055
C= 5509.951373978198
Xcur= [1, 5, 2, 9, 3, 4, 10, 6, 8, 7, 1] Zcur= 48425.825659963055
i= 3
Xcount= [1, 5, 2, 7, 3, 4, 10, 6, 8, 9, 1] Zcount= 51200.694597931084
C= 2774.868937968029
Xcur= [1, 5, 2, 7, 3, 4, 10, 6, 8, 9, 1] Zcur= 51200.694597931084
Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 42915.87428598486
1.7935469150543213
Tcur= 3610
i= 1
Xcount= [1, 5, 2, 7, 8, 6, 3, 4, 10, 9, 1] Zcount= 46515.3461735434
Z-Zcur= -4685.348424387681
Xcur= [1, 5, 2, 7, 8, 6, 3, 4, 10, 9, 1] Zcur= 46515.3461735434
i= 2
Xcount= [1, 5, 2, 7, 8, 9, 3, 4, 10, 6, 1] Zcount= 50282.160553840375
C= 3766.8143802969716
i= 3
Xcount= [1, 5, 2, 8, 7, 6, 10, 4, 3, 9, 1] Zcount= 48733.29677667196
C= 2217.9506031285564
Xcur= [1, 5, 2, 8, 7, 6, 10, 4, 3, 9, 1] Zcur= 48733.29677667196
Xbest= [1, 9, 3, 4, 10, 6, 8, 7, 2, 5, 1] Zbest= 429

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,ganttColor,popupText,startElevMeters,endElevMeters,wayname,waycategory,surface,waytype,steepness,tollway
0,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,0.000000,42.970124,-78.639963,0,5.895919,...,darkgray,None,225.8,225.7,-,No category,Asphalt,Street,0,False
1,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,5.895919,42.970408,-78.639858,0,24.800000,...,darkgray,None,225.7,225.4,-,No category,Asphalt,Street,0,False
2,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,24.800000,42.971338,-78.640075,0,26.658000,...,darkgray,None,225.4,225.4,"Main Street, NY 5",No category,Asphalt,State Road,0,False
3,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,26.658000,42.971260,-78.640441,0,30.446964,...,darkgray,None,225.4,225.2,"Main Street, NY 5",No category,Asphalt,State Road,0,False
4,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,30.446964,42.971100,-78.641187,0,30.818380,...,darkgray,None,225.2,225.1,"Main Street, NY 5",No category,Asphalt,State Road,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,5208.467360,42.971555,-78.639066,0,5209.897826,...,darkgray,None,225.6,225.7,"Main Street, NY 5",No category,Asphalt,State Road,0,False
1241,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,5209.897826,42.971494,-78.639347,0,5212.658533,...,darkgray,None,225.7,225.8,"Main Street, NY 5",No category,Asphalt,State Road,0,False
1242,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,5212.658533,42.971378,-78.639890,0,5213.600000,...,darkgray,None,225.8,225.8,"Main Street, NY 5",No category,Asphalt,State Road,0,False
1243,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,5213.600000,42.971338,-78.640075,0,5232.504081,...,darkgray,None,225.8,225.5,-,No category,Asphalt,Street,0,False


In [20]:
vrv.createLeaflet(arcs=X_best,nodes=nodesDF)